In [ ]:
import os
import sys
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
import string
import spacy
import random
from pathlib import Path
import re
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import re
import spacy
from nltk.corpus import sentiwordnet as swn
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [ ]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [ ]:
data= pd.read_csv('/content/drive/MyDrive/Dataset/dataset.csv', encoding="latin1")
data.head()

,Sentiment,SentimentText
0,Negative,is so sad for my APL friend.............
1,Negative,I missed the New Moon trailer...
2,Positive,omg its already 7:30 :O
3,Negative,.. Omgaga. Im sooo im gunna CRy. I've been at...
4,Negative,i think mi bf is cheating on me!!! T_T


In [ ]:
data['Sentiment'].unique()

array(['Negative', 'Positive', nan], dtype=object)

In [ ]:
data['Sentiment'].replace({"1.0":1,"0.0":0},inplace=True)

In [ ]:
data["Sentiment"].unique()

array(['Negative', 'Positive', nan], dtype=object)

In [ ]:
Edited_Tweets = data['SentimentText'].copy()
data['Tweets_without_stopwords'] = Edited_Tweets

In [ ]:
# Function to preprocess tweets 
def preprocess_tweet_data(data,name):
    # Proprocessing the data
    data[name]=data[name].str.lower()
    # Code to remove the Hashtags from the text
    data[name]=data[name].apply(lambda x:re.sub(r'\B#\S+','',x))
    # Code to remove the links from the text
    data[name]=data[name].apply(lambda x:re.sub(r"http\S+", "", x))
    # Code to remove the Special characters from the text 
    data[name]=data[name].apply(lambda x:' '.join(re.findall(r'\w+', x)))
    # Code to substitute the multiple spaces with single spaces
    data[name]=data[name].apply(lambda x:re.sub(r'\s+', ' ', x, flags=re.I))
    # Code to remove all the single characters in the text
    data[name]=data[name].apply(lambda x:re.sub(r'\s+[a-zA-Z]\s+', '', x))
    # Remove the twitter handlers
    data[name]=data[name].apply(lambda x:re.sub('@[^\s]+','',x))

# Function to tokenize and remove the stopwords    
def rem_stopwords_tokenize(data,name):
      
    def getting(sen):
        example_sent = sen
        
        filtered_sentence = [] 

        stop_words = set(stopwords.words('english')) 

        word_tokens = word_tokenize(example_sent) 
        
        filtered_sentence = [w for w in word_tokens if not w in stop_words] 
        
        return filtered_sentence
    # Using "getting(sen)" function to append edited sentence to data
    x=[]
    for i in data[name].values:
        x.append(getting(i))
    data[name]=x

In [ ]:
lemmatizer = WordNetLemmatizer()
def Lemmatization(data,name):
    def getting2(sen):
        
        example = sen
        output_sentence =[]
        word_tokens2 = word_tokenize(example)
        lemmatized_output = [lemmatizer.lemmatize(w) for w in word_tokens2]
        
        # Remove characters which have length less than 2  
        without_single_chr = [word for word in lemmatized_output if len(word) > 2]
        # Remove numbers
        cleaned_data_title = [word for word in without_single_chr if not word.isnumeric()]
        
        return cleaned_data_title
    x=[]
    for i in data[name].values:
        x.append(getting2(i))
    data[name]=x

In [ ]:
def make_sentences(data,name):
    data[name]=data[name].apply(lambda x:' '.join([i+' ' for i in x]))
    # Removing double spaces if created
    data[name]=data[name].apply(lambda x:re.sub(r'\s+', ' ', x, flags=re.I))

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
# Using the preprocessing function to preprocess the tweets
preprocess_tweet_data(data,'Tweets_without_stopwords')
# Using tokenizer and removing the stopwords
rem_stopwords_tokenize(data,'Tweets_without_stopwords')
# Converting all the texts back to sentences
make_sentences(data,'Tweets_without_stopwords')

#Edits After Lemmatization
final_Edit = data['Tweets_without_stopwords'].copy()
data["After_lemmatization"] = final_Edit

# Using the Lemmatization function to lemmatize the tweets
Lemmatization(data,'After_lemmatization')
# Converting all the texts back to sentences
make_sentences(data,'After_lemmatization')

TypeError: ignored

In [ ]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
import nltk
import ssl
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords


In [ ]:
nltk.download('averaged_perceptron_tagger')
nltk.download('sentiwordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!


True

In [ ]:
pos=neg=obj=count=0

postagging = []

for tweet in data['After_lemmatization']:
    li = word_tokenize(tweet)
    postagging.append(nltk.pos_tag(li))

data['pos_tags'] = postagging

def penn_to_wn(tag):
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
         return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None


# Returns list of pos-neg and objective score. But returns empty list if not present in senti wordnet.
def get_sentiment(word,tag):
    wn_tag = penn_to_wn(tag)
    
    if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
        return []

    #Lemmatization
    lemma = lemmatizer.lemmatize(word, pos=wn_tag)
    if not lemma:
        return []

    synsets = wn.synsets(word, pos=wn_tag)
    if not synsets:
        return []

    # Take the first sense, the most common
    synset = synsets[0]
    swn_synset = swn.senti_synset(synset.name())

    return [synset.name(), swn_synset.pos_score(),swn_synset.neg_score(),swn_synset.obj_score()]

    pos=neg=obj=count=0
  
senti_score = []

for pos_val in data['pos_tags']:
    senti_val = [get_sentiment(x,y) for (x,y) in pos_val]
    for score in senti_val:
        try:
            pos = pos + score[1]  #positive score is stored at 2nd position
            neg = neg + score[2]  #negative score is stored at 3rd position
        except:
            continue
    senti_score.append(pos - neg)
    pos=neg=0    
data['senti_score'] = senti_score
print(data['senti_score'])

data.head()

KeyError: ignored

In [ ]:
overall=[]
for i in range(len(data)):
    if data['senti_score'][i]>= 0.05:
        overall.append('Positive')
    elif data['senti_score'][i]<= -0.05:
        overall.append('Negative')
    else:
        overall.append('Neutral')
data['Overall Sentiment']=overall
data.head()

,ItemID,Sentiment,SentimentSource,SentimentText,Tweets_without_stopwords,After_lemmatization,pos_tags,senti_score,Overall Sentiment
0,1,0,Sentiment140,is so sad for my APL frie...,sad apl friend,sad apl friend,"[(sad, JJ), (apl, NN), (friend, NN)]",-0.500,Negative
1,2,0,Sentiment140,I missed the New Moon trail...,missed new moon trailer,missed new moon trailer,"[(missed, VBN), (new, JJ), (moon, NN), (traile...",0.375,Positive
2,3,1,Sentiment140,omg its already 7:30 :O,omg already 7 30,omg already,"[(omg, NNS), (already, RB)]",0.125,Positive
3,4,0,Sentiment140,.. Omgaga. Im sooo im gunna CRy. I'...,omgaga im sooo im gunna cryve dentist since 11...,omgaga sooo gunna cryve dentist since 11was su...,"[(omgaga, JJ), (sooo, NN), (gunna, NN), (cryve...",0.000,Neutral
4,5,0,Sentiment140,i think mi bf is cheating on me!!! ...,think mi bf cheating t_t,think cheating t_t,"[(think, NN), (cheating, VBG), (t_t, NN)]",0.000,Neutral


In [ ]:
data.to_csv("/drive/MyDrive/sentiment-cnn/cleaned_data.csv")

In [ ]:
# Read file from gdrive
import pandas as pd
import numpy as np
data=pd.read_csv('/drive/MyDrive/sentiment-cnn/cleaned_data.csv')

In [ ]:
data.dropna(subset=["After_lemmatization"], inplace=True)

In [ ]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

In [ ]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(data['After_lemmatization'].values)
word_index = tokenizer.word_index

In [ ]:
from keras.preprocessing.sequence import pad_sequences
X = tokenizer.texts_to_sequences(data['After_lemmatization'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)

In [ ]:
#Y = pd.get_dummies(data['sentiment']).values
from sklearn.preprocessing import LabelEncoder
mlb = LabelEncoder()
sentiment = data['Sentiment'].to_numpy()
mlb.fit(sentiment)
Y = mlb.transform(sentiment)

In [ ]:
# Train-Test Split
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.1, random_state=42)

In [ ]:
## build training vocabulary and get maximum training sentence length and total number of words training data
all_training_words = ''.join([ word for tokens in train["After_lemmatization"] for word in tokens])
training_sentence_lengths = [len(tokens) for tokens in train["After_lemmatization"]]
TRAINING_VOCAB = sorted([(set(all_training_words))])
print("%s words total, with a vocabulary size of %s" % (len(all_training_words), len(TRAINING_VOCAB)))

179768 words total, with a vocabulary size of 1


In [ ]:
## Build testing vocabulary and get maximum testing sentence length and total number of words in testing data

all_test_words = ''.join([word for tokens in test["After_lemmatization"] for word in tokens])
test_sentence_lengths = [len(tokens) for tokens in test["After_lemmatization"]]
TEST_VOCAB = sorted(list(set(all_test_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_test_words), len(TEST_VOCAB)))
print("Max sentence length is %s" % max(test_sentence_lengths))

19745 words total, with a vocabulary size of 41
Max sentence length is 117


In [ ]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=len(TRAINING_VOCAB), lower=True, char_level=False)

In [ ]:
!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz" -O "/drive/MyDrive/GoogleNews-vectors-negative300.bin.gz"

--2022-03-11 02:46:48--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.4.110
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.4.110|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [ ]:
from gensim.models import KeyedVectors
word2vec_path = '/drive/MyDrive/GoogleNews-vectors-negative300.bin.gz'
word2vec = KeyedVectors.load_word2vec_format(word2vec_path, binary=True, limit=100000)

In [ ]:
# Tokenization
tokenizer.fit_on_texts(train["After_lemmatization"].tolist())
training_sequences = tokenizer.texts_to_sequences(train["After_lemmatization"].tolist())
testing_sequences = tokenizer.texts_to_sequences(test["After_lemmatization"].tolist())
train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))

Found 9669 unique tokens.


In [ ]:
def get_weight_matrix(embedding, vocab):
	# total vocabulary size plus 0 for unknown words
	vocab_size = len(vocab) + 1
	# define weight matrix dimensions with all 0
	weight_matrix = np.zeros((vocab_size, 100))
	# step vocab, store vectors using the Tokenizer's integer mapping
	for word, i in vocab.items():
		vector = embedding.get(word)
		if vector is not None:
			weight_matrix[i] = vector
	return weight_matrix

In [ ]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 40
# This is fixed.
EMBEDDING_DIM = 300

In [ ]:
from keras.preprocessing.sequence import pad_sequences
train_cnn_data = pad_sequences(training_sequences, 
                               maxlen=MAX_SEQUENCE_LENGTH)
test_cnn_data = pad_sequences(testing_sequences , 
                               maxlen=MAX_SEQUENCE_LENGTH)

In [ ]:
train_embedding_weights = np.zeros((len(train_word_index)+1, 
 EMBEDDING_DIM))

In [ ]:
# Word2Vec word Embedding
for word,index in train_word_index.items():
    train_embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(EMBEDDING_DIM)
print(train_embedding_weights.shape)

(9670, 300)


In [ ]:
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse=False)
ytrain = one_hot_encoder.fit_transform(train["Overall Sentiment"].values.reshape(-1, 1))
cnn_xtrain = train_cnn_data

ytest = one_hot_encoder.transform(test["Overall Sentiment"].values.reshape(-1, 1))
cnn_xtest = test_cnn_data

In [ ]:
# define model
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Embedding
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

def create_model(embeddings, max_sequence_length, num_words, embedding_dim, labels_index):
  embedding_layer = Embedding(num_words,
                              embedding_dim,
                              weights=[embeddings],
                              input_length=max_sequence_length,
                              trainable=False)
  model = Sequential()
  model.add(embedding_layer)
  model.add(Conv1D(filters=30, kernel_size=5, padding = "valid", activation='relu'))
  model.add(Conv1D(10, kernel_size = 3, padding='valid', activation='relu', strides=1))

  model.add(Flatten())
  model.add(Dense(128, activation='relu'))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(3, activation='softmax'))
  print(model.summary())
  return model

In [ ]:
model = create_model(train_embedding_weights, 
                MAX_SEQUENCE_LENGTH, 
                len(train_word_index)+1, 
                EMBEDDING_DIM, 
                3)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 40, 300)           2901000   
                                                                 
 conv1d_4 (Conv1D)           (None, 36, 30)            45030     
                                                                 
 conv1d_5 (Conv1D)           (None, 34, 10)            910       
                                                                 
 flatten_2 (Flatten)         (None, 340)               0         
                                                                 
 dense_6 (Dense)             (None, 128)               43648     
                                                                 
 dense_7 (Dense)             (None, 64)                8256      
                                                                 
 dense_8 (Dense)             (None, 3)                

In [ ]:
filepath = "./models/cnn-{epoch:02d}-{loss:0.3f}-{val_loss:0.3f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor="loss", verbose=1, save_best_only=True, mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=0.000001)
model.fit(cnn_xtrain, ytrain, batch_size=200, epochs=50, validation_split=0.1, shuffle=True, callbacks=[checkpoint, reduce_lr])

Epoch 1/50
21/21 [==============================] - ETA: 0s - loss: 1.0964 - accuracy: 0.4488
Epoch 1: loss improved from inf to 1.09640, saving model to ./models/cnn-01-1.096-1.095.hdf5
21/21 [==============================] - 2s 30ms/step - loss: 1.0964 - accuracy: 0.4488 - val_loss: 1.0948 - val_accuracy: 0.4340 - lr: 0.0010
Epoch 2/50
16/21 [=====================>........] - ETA: 0s - loss: 1.0925 - accuracy: 0.4559
Epoch 2: loss improved from 1.09640 to 1.09194, saving model to ./models/cnn-02-1.092-1.091.hdf5
21/21 [==============================] - 0s 12ms/step - loss: 1.0919 - accuracy: 0.4592 - val_loss: 1.0915 - val_accuracy: 0.4340 - lr: 0.0010
Epoch 3/50
16/21 [=====================>........] - ETA: 0s - loss: 1.0887 - accuracy: 0.4556
Epoch 3: loss improved from 1.09194 to 1.08793, saving model to ./models/cnn-03-1.088-1.089.hdf5
21/21 [==============================] - 0s 12ms/step - loss: 1.0879 - accuracy: 0.4592 - val_loss: 1.0886 - val_accuracy: 0.4340 - lr: 0.0010
Ep

In [ ]:
loss, acc = model.evaluate(cnn_xtest, ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))

Test Accuracy: 50.503016


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import svm
from sklearn.metrics import accuracy_score

In [ ]:
bow_vectorizer = CountVectorizer(max_df=0.90, min_df=2, stop_words='english')
bow_train = bow_vectorizer.fit_transform(train["After_lemmatization"])
bow_test = bow_vectorizer.transform(test["After_lemmatization"])

In [ ]:
def concat_softmax(model, data, bow):
  dense_bow = bow.todense()
  output = model.predict(data)
  result = np.concatenate([dense_bow, output], axis=1)
  return result

In [ ]:
svm_xtrain = concat_softmax(model, cnn_xtrain, bow_train)
svm_xtest = concat_softmax(model, cnn_xtest, bow_test)

In [ ]:
svc = svm.SVC(kernel='rbf', gamma=0.5, C=0.1, probability=True).fit(svm_xtrain, train["Overall Sentiment"].values)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


In [ ]:
pred = svc.predict(svm_xtest)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


In [ ]:
accuracy_score(test["Overall Sentiment"], pred)

0.5935613682092555

In [ ]:
def concat_softmax_svm(cnn_model, cnn_data, svm_model, svm_data):
  svm_output = svm_model.predict_proba(svm_data)
  cnn_output = cnn_model.predict(cnn_data)
  result = np.concatenate([cnn_output, svm_output], axis=1)
  return result

In [ ]:
final_svm_xtrain = concat_softmax_svm(model, cnn_xtrain, svc, svm_xtrain)
final_svm_xtest = concat_softmax_svm(model, cnn_xtest, svc, svm_xtest)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


In [ ]:
final_svm_xtest.shape

(497, 6)

In [ ]:
final_svc = svm.SVC(kernel='rbf', gamma=0.5, C=0.1, probability=True).fit(final_svm_xtrain, train["Overall Sentiment"].values)

In [ ]:
final_pred = final_svc.predict(final_svm_xtest)
accuracy_score(test["Overall Sentiment"], final_pred)

0.6639839034205232

In [ ]:
from sklearn.metrics import f1_score
f1_score(test["Overall Sentiment"], final_pred, average="macro")

0.634015063670394